In [4]:
import os
import json
import ftplib
import datetime as dt

with open("credentials/passwords.json", "r") as file:
    passwords = json.load(file)

class InvalidPathError(Exception):
    pass

class Bluehost:
    def __init__(self):
        self.ftp = None

    def __enter__(self):
        self.ftp = self.login()
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        try:
            if self.ftp:
                self.ftp.quit()
        finally:
            self.ftp = None
        return False

    def login(self):
        ftp_server = 'ftp.luh.sgg.mybluehost.me'
        ftp_username = 'mpuche2@luh.sgg.mybluehost.me'
        ftp_password = passwords["BLUEHOST"]
        self.ftp = ftplib.FTP(ftp_server)
        try:
            self.ftp.login(ftp_username, ftp_password)
            welcome_message = self.ftp.getwelcome()
            if welcome_message:
                print("===")
                print("FTP connection established successfully!")
                print("Welcome message:", welcome_message)
                print("===")
            else:
                print("FTP connection failed. Check your credentials.")
        except ftplib.all_errors as e:
            print("FTP connection failed:", e)
            self.ftp.quit()
        finally:
            return self.ftp

    def logout(self):
        if hasattr(self, 'ftp'):
            self.ftp.quit() 

    def file_exists(self, path):
        try:
            self.ftp.size(path)
            return True
        except ftplib.error_perm:
            return False

    def dir_exists(self, path):
        try:
            self.ftp.nlst(path)
            return True
        except ftplib.error_perm:
            return False

    def listdir_recursive_only_files(self, path = os.path.join("")):
        try:
            listdir = self.ftp.nlst(path)
        except Exception as e:
            print(f"Error listing dir {path}: {e}")
            return []

        cleaned_listdir = []
        for x in listdir:
            if x[0] == "." or x[-1] == ".":
                continue
            elif x.count('.') > 0:
                cleaned_listdir.append(x)
            else:
                cleaned_listdir.extend(self.listdir_recursive_only_files(x))

        cleaned_listdir = sorted(cleaned_listdir)
        return cleaned_listdir

    def listdir(self, path = os.path.join("")):
        try:
            listdir = self.ftp.nlst(path)
        except Exception as e:
            print(f"Error listing dir {path}: {e}")
            return []

        cleaned_listdir = []
        for x in listdir:
            if x == "." or x[-1] == "." or x == "cgi-bin":
                continue
            cleaned_listdir.append(x)

        cleaned_listdir = sorted(cleaned_listdir)
        return cleaned_listdir

    def upload_file(self, local_path, remote_path, overwrite=False):
        local_dir = os.path.dirname(local_path)
        remote_dir = os.path.dirname(remote_path)
        local_dir = local_dir if local_dir != "" else os.path.join(".")
        remote_dir = remote_dir if local_dir != "" else os.path.join(".")
        if not os.path.exists(local_dir):
            raise InvalidPathError(f"Local path dir doesn't exist: {local_dir}")
        if not os.path.exists(local_path):
            raise InvalidPathError(f"Local path doesn't exist: {local_path}")
        if not self.dir_exists(remote_dir):
            raise InvalidPathError(f"Remote dir doesn't exist: {remote_dir}")
        if not overwrite and self.file_exists(remote_path):
            raise InvalidPathError(f"Remote file already exists: {remote_path}")
        with open(local_path, 'rb') as file:
            response = self.ftp.storbinary(f"STOR {remote_path}", file)
        print("---")
        print(f"filename: {os.path.basename(local_path)}")
        print(f"local path: {local_path}")
        print(f"remote path: {remote_path}")
        print(f"response: {response}")

    def download_file(self, remote_path, local_path, overwrite=False):
        local_dir = os.path.dirname(local_path)
        remote_dir = os.path.dirname(remote_path)
        local_dir = local_dir if local_dir != "" else os.path.join(".")
        remote_dir = remote_dir if local_dir != "" else os.path.join(".")
        if not os.path.exists(local_dir):
            raise InvalidPathError(f"Local dir doesn't exist: {local_dir}")
        if not self.dir_exists(remote_dir):
            raise InvalidPathError(f"Remote dir doesn't exist: {remote_dir}")
        if not self.file_exists(remote_path):
            raise InvalidPathError(f"Remote file doesn't exists: {remote_path}")
        if os.path.exists(local_path) and overwrite == False:
            raise InvalidPathError(f"Local file already exist: {local_path}")
        if os.path.exists(local_path) and overwrite == "rename":
            os.rename(local_path, local_path + "_OLD_" + dt.datetime.now().strftime("_%Y%m%d%H%M%S"))
        with open(local_path, 'wb') as file:
            response = self.ftp.retrbinary('RETR ' + remote_path, file.write)
        print("---")
        print(f"filename: {os.path.basename(local_path)}")
        print(f"local path: {local_path}")
        print(f"remote path: {remote_path}")
        print(f"response: {response}")

In [5]:
def download_transcripts_from_bluehost():
    with Bluehost() as bh:
        ...
        # contents = bh.listdir(os.path.join(".", "transcriptions", "books", book))
        # print(contents)
        # for book in ["B001", "B002", "B009"]:
        #     path = os.path.join(".", "transcriptions", "books", book, f"{book}_TRANS_ALL.txt")
        #     bh.download_file(
        #         remote_path = path,
        #         local_path = path,
        #         overwrite=True
        #     )

def update_website_in_bluehost():
    with Bluehost() as bh:
        for filename in ["index.html", "style.css", "script_slow.js", "script_fast.js"]:
            path = os.path.join(".", filename)
            bh.upload_file(
                remote_path = path,
                local_path = path,
                overwrite=True
            )

def update_website_dev_in_bluehost():
    filenames = ["index.html", "script.js", "style.css"]
    with Bluehost() as bh:
        for filename in filenames:
            bh.upload_file(
                remote_path = os.path.join(".", "tester", filename),
                local_path = os.path.join(".", filename),
                overwrite=True
            )

# def create_folder():
#     bh = Bluehost()
#     ftp = bh.login()
#     ftp.mkd('filters')
#     ftp.close()

# def upload_file():
#     path = os.path.join(".", "filters", "filters.txt")
#     with Bluehost() as bh:
#         bh.upload_file(
#             local_path = path,
#             remote_path = path,
#             overwrite="rename",
#         )

# def deleteme01():
#     text = ""
#     for book in ["B001", "B002", "B009"]:
#         path = os.path.join(".", "transcriptions", "books", book, f"{book}_TRANS_ALL.txt")
#         assert os.path.exists(path)
#         with open(path, "r") as file:
#             text += file.read()

#     ans = ""
#     lines = text.split("\n")
#     for line in lines:
#         if line.startswith("B0"):
#             ans += "[o] " + line + "\n"

#     with open("deleteme.txt", "w") as file:
#         file.write(ans)

# def deleteme02():
#     with open("./filters/filters.txt", "r") as file:
#         text = file.read(ans)

#     lines = text.split("\n")
#     for line in lines:
#         if line.startswith("[0]"):
#             ans += line + "\n"

#     with open("./filters/included.txt", "w") as file:
#         text = file.write(ans)

In [27]:
# update_website_dev_in_bluehost()

===
FTP connection established successfully!
Welcome message: 220---------- Welcome to Pure-FTPd [privsep] [TLS] ----------
220-You are user number 1 of 150 allowed.
220-Local time is now 09:24. Server port: 21.
220-IPv6 connections are also welcome on this server.
220 You will be disconnected after 15 minutes of inactivity.
===
---
filename: index.html
local path: ./index.html
remote path: ./tester/index.html
response: 226-File successfully transferred
226 0.130 seconds (measured here), 10.66 Kbytes per second
---
filename: script.js
local path: ./script.js
remote path: ./tester/script.js
response: 226-File successfully transferred
226 0.265 seconds (measured here), 76.29 Kbytes per second
---
filename: style.css
local path: ./style.css
remote path: ./tester/style.css
response: 226-File successfully transferred
226 0.140 seconds (measured here), 20.53 Kbytes per second


In [7]:
# with Bluehost() as bh:
#     folders = ["questions", "audio"]
#     for folder in folders:
#         filenames = os.listdir(os.path.join(".", "data", folder))
#         for filename in filenames:
#             ...
#             # local_path = os.path.join(".", "data", folder, filename)         
#             # remote_path = os.path.join(".", "tester", "data", folder, filename)
#             # print(local_path)
#             # print(remote_path)
#             # print()
#             # bh.upload_file(local_path, remote_path , overwrite = True)
#         path = os.path.join(".", "tester", "data", folder)
#         print(path)
#         content = bh.ftp.dir(path)

# print(content)

In [26]:
def upload_questions():
    filename = "UK_Today.json"
    dir_path_local = os.path.join(".", "data", "questions")         
    dir_path_remote = os.path.join(".", "tester", "data", "questions")
    file_path_local = os.path.join(dir_path_local, filename)
    file_path_remote = os.path.join(dir_path_remote, filename)
    os.path.exists(file_path_local)

    with Bluehost() as bh:
        bh.upload_file(file_path_local, file_path_remote , overwrite = False)
        contents = bh.ftp.dir(dir_path_remote)

    print(contents)



===
FTP connection established successfully!
Welcome message: 220---------- Welcome to Pure-FTPd [privsep] [TLS] ----------
220-You are user number 1 of 150 allowed.
220-Local time is now 09:23. Server port: 21.
220-IPv6 connections are also welcome on this server.
220 You will be disconnected after 15 minutes of inactivity.
===
---
filename: UK_Today.json
local path: ./data/questions/UK_Today.json
remote path: ./tester/data/questions/UK_Today.json
response: 226-File successfully transferred
226 0.772 seconds (measured here), 0.53 Mbytes per second
drwxr-xr-x    2 luhsggmy   luhsggmy         4096 Jan 16 09:23 .
drwxr-xr-x    4 luhsggmy   luhsggmy         4096 Jan 14 23:06 ..
-rw-r--r--    1 luhsggmy   luhsggmy       215024 Jan 14 23:23 ALL.json
-rw-r--r--    1 luhsggmy   luhsggmy         7252 Jan 14 23:23 C001.json
-rw-r--r--    1 luhsggmy   luhsggmy        11074 Jan 14 23:23 C002.json
-rw-r--r--    1 luhsggmy   luhsggmy       386438 Jan 14 23:23 Famous.json
-rw-r--r--    1 luhsggmy   

In [8]:
# with Bluehost() as bh:
#     path = os.path.join(".", "tester")
#     print(path)
#     content = bh.ftp.dir(path)

In [9]:
# update_website_dev_in_bluehost()
# update_website_in_bluehost()

In [10]:
# # update_website_in_bluehost()

# !git add .
# !git commit -m message
# !git push
# !git status

# !git add .
# !git commit -m message
# !git push
# !git status

In [11]:
# with Bluehost() as bh:
#     # bh.ftp.rename('./audio/books/B014', './audio/books/DELETEME_B014')
#     bh.ftp.mkd('./audio/books/B014')
#     path_text = os.path.join(".", "text", "books")
#     path_tran = os.path.join(".", "transcriptions", "books")
#     path_audio = os.path.join(".", "audio", "books")
#     contents = bh.ftp.nlst(path_audio)
#     print(contents)

In [12]:
# for i in range(14, 15):
#     book = "B0" + str(i).zfill(2)
#     # os.mkdir(os.path.join(".", "text", "books", book))
#     # os.mkdir(os.path.join(".", "transcriptions", "books", book))
#     with Bluehost() as bh:
#         bh.download_file(
#             remote_path= os.path.join(".", "text", "books", book, f'{book}_TEXTS_ALL.txt'),
#             local_path = os.path.join(".", "text", "books", book, f'{book}_TEXTS_ALL.txt'),
#             overwrite = "rename"
#         )
#         bh.download_file(
#             remote_path= os.path.join(".", "transcriptions", "books", book, f'{book}_TRANS_ALL.txt'),
#             local_path = os.path.join(".", "transcriptions", "books", book, f'{book}_TRANS_ALL.txt'),
#             overwrite = "rename"
#         )

In [13]:
# book = "B014"
# os.mkdir(os.path.join(".", "text", "books", book))
# os.mkdir(os.path.join(".", "transcriptions", "books", book))
# with Bluehost() as bh:
#     bh.download_file(
#         remote_path= os.path.join(".", "text", "books", book, f'{book}_TEXTS_ALL.txt'),
#         local_path = os.path.join(".", "text", "books", book, f'{book}_TEXTS_ALL.txt'),
#         overwrite = "rename"
#     )
#     bh.download_file(
#         remote_path= os.path.join(".", "transcriptions", "books", book, f'{book}_TRANS_ALL.txt'),
#         local_path = os.path.join(".", "transcriptions", "books", book, f'{book}_TRANS_ALL.txt'),
#         overwrite = "rename"
#     )

In [14]:
# path = os.path.join(".", "text", "books", "B004")
# with Bluehost() as bh:
#     contents = bh.listdir(path)
# print(contents)

In [15]:
# path = os.path.join(".", "filters", "filters.txt")
# with Bluehost() as bh:
#     bh.upload_file(
#         remote_path = path,
#         local_path = path,
#         # overwrite=True
#     )

In [16]:
# bh = Bluehost()
# ftp = bh.login()

In [17]:
# path = os.path.join(".", "text", "books")
# bh.listdir(path)

In [18]:
# book = "B009"
# path = os.path.join(".", "text", "books", book, f"{book}_TEXTS_ALL.txt")
# bh.upload_file(
#     local_path = path,
#     remote_path = path
# )

In [19]:
# bh.logout()

In [20]:
# upload_file()

In [21]:
def generate_B001():
    path = os.path.join(".", "filters", "filters.txt")
    with open(path, "r") as file:
        filters = file.read()

    filters = filters.strip().split("\n")
    filters = [filter for filter in filters if "[o] B001" in filter]
    filters = [filter[4:4+8] for filter in filters]

    path = os.path.join(".", "transcriptions", "books", "B001", "B001_TRANS_ALL.txt")
    with open(path, "r") as file:
        lines = file.read().strip().split("\n")

    book = ""
    include = True
    for line in lines:
        if line.startswith("B001"):
            if line[0: 8] in filters:
                include = True
            else:
                include =False
        if include == True:
            book += line + "\n"

    with open(os.path.join(".", "B001.txt"), "w") as file:
        file.write(book)